Useful links:

- https://eprint.iacr.org/2019/953.pdf
- https://blog.lambdaclass.com/all-you-wanted-to-know-about-plonk/
- https://risencrypto.github.io/PLONKPerm/
- https://www.cryptologie.net/article/594/whats-happening-in-the-round-5-of-plonk/
- https://trapdoortech.medium.com/zkp-plonk-algorithm-introduction-834556a32a
- https://xiaohuiliu.medium.com/how-plonk-works-part-2-1072dcd7634a
- https://medium.com/coinmonks/under-the-hood-of-zksnarks-plonk-protocol-part-1-34bc406d8303
- https://research.metastate.dev/plonk-by-hand-part-1/

In [1]:
import galois
import numpy as np

from utils import generator1, generator2, curve_order, normalize, validate_point, numbers_to_hash

In [2]:
p = curve_order
Fp = galois.GF(p)

In [ ]:
# We have 7 gates, next power of 2 is 8
n = 7
n = 2**int(np.ceil(np.log2(n)))
assert n & n - 1 == 0, "n must be a power of 2"

# Find primitive root of unity (generator)
omega = Fp.primitive_root_of_unity(n)
assert omega**(n) == 1, f"omega (ω) {omega} is not a root of unity"

roots = Fp([omega**i for i in range(n)])

### Gate constraints

In [ ]:
def pad_array(a, n):
    return a + [0]*(n - len(a))


def to_galois_array(vector, field):
    # normalize to positive values
    a = [x % field.order for x in vector]
    return field(a)

# gate vectors
ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, p-1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 5]
qo = [p-1, p-1, p-1, p-1, p-1, p-1, p-1]

# pad vectors to length n
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)

ql = to_galois_array(ql, Fp)
qr = to_galois_array(qr, Fp)
qm = to_galois_array(qm, Fp)
qc = to_galois_array(qc, Fp)
qo = to_galois_array(qo, Fp)

# public input
pi = [0, 0, 0, 0, 0, 0, -2]
pi = pad_array(pi, n)
pi = to_galois_array(pi, Fp)

### Permutation check

In [ ]:
ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

k1 = 2
k2 = 4
c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for r in c_roots:
    assert int(r) not in check, f"Duplicate root {r} in {c_roots}"
    check.add(int(r))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

In [ ]:
def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if isinstance(v, list) else v
    return galois.lagrange_poly(x, y)

QL = to_poly(roots, ql, Fp)
QR = to_poly(roots, qr, Fp)
QM = to_poly(roots, qm, Fp)
QC = to_poly(roots, qc, Fp)
QO = to_poly(roots, qo, Fp)

PI = to_poly(roots, pi, Fp) # degree n-1

for i in range(n):
    assert QL(roots[i]) == ql[i], f"QL({roots[i]}) != ql[{i}]"
    assert QR(roots[i]) == qr[i], f"QR({roots[i]}) != qr[{i}]"
    assert QM(roots[i]) == qm[i], f"QM({roots[i]}) != qm[{i}]"
    assert QC(roots[i]) == qc[i], f"QC({roots[i]}) != qc[{i}]"
    assert QO(roots[i]) == qo[i], f"QO({roots[i]}) != qo[{i}]"
    assert PI(roots[i]) == pi[i], f"PI({roots[i]}) != pi[{i}]"

In [ ]:
S1 = to_poly(roots, sigma1, Fp)
S2 = to_poly(roots, sigma2, Fp)
S3 = to_poly(roots, sigma3, Fp)

I1 = to_poly(roots, c1_roots, Fp)
I2 = to_poly(roots, c2_roots, Fp)
I3 = to_poly(roots, c3_roots, Fp)

for i in range(n):
    assert S1(roots[i]) == sigma1[i], f"S1({roots[i]}) != sigma1[{i}]"
    assert S2(roots[i]) == sigma2[i], f"S2({roots[i]}) != sigma2[{i}]"
    assert S3(roots[i]) == sigma3[i], f"S3({roots[i]}) != sigma3[{i}]"
    assert I1(roots[i]) == c1_roots[i], f"I1({roots[i]}) != c1_roots[{i}]"
    assert I2(roots[i]) == c2_roots[i], f"I2({roots[i]}) != c2_roots[{i}]"
    assert I3(roots[i]) == c3_roots[i], f"I3({roots[i]}) != c3_roots[{i}]"


In [ ]:
Zh = galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=Fp)
for x in roots:
    assert Zh(x) == 0

### Trusted setup

$CRS = \left\langle [\tau^0]_{G_1}, [\tau^1]_{G_1}], ..., [\tau^{n+5}]_{G_1} \right\rangle, [\tau]_{G_2}\\$

In [ ]:
G1 = generator1()
G2 = generator2()

class SRS:
    """Trusted Setup Class aka Structured Reference String"""
    def __init__(self, tau, n):
        self.tau = tau
        self.tau1 = [G1 * int(tau)**i for i in range(0, (n + 5) + 1)]
        self.tau2 = G2 * int(tau)

    def __str__(self):
        s = f"tau: {self.tau}\n"
        s += "".join([f"[tau^{i}]G1: {str(normalize(point))}\n" for i, point in enumerate(self.tau1)])
        s += f"[tau]G2: {str(normalize(self.tau2))}\n"
        return s

def new_call(self, at, **kwargs):
    if isinstance(at, SRS):
        coeffs = self.coeffs[::-1]
        result = at.tau1[0] * coeffs[0]
        for i in range(1, len(coeffs)):
            result += at.tau1[i] * coeffs[i]
        return result

    return galois.Poly.original_call(self, at, **kwargs)

galois.Poly.original_call = galois.Poly.__call__
galois.Poly.__call__ = new_call

tau = SRS(Fp.Random(), n)

### Proving

#### Statement

I know a quotient polynomial $T(x)$ such that $T(x) \cdot Z_H(x) = P(x)$. Polynomial $P(x)$ evaluates to 0 at all $H = \{\omega^i\}_{i\in[n]}$.

$P(x)$ consists of the following components:
- gates (all witnesses are computed correctly): $A(x)Q_L(x) + B(x)Q_R(x) + C(x)Q_O(x) + A(x)B(x)Q_M(x) + Q_C(x) + PI(x) = 0$
- permutations (all witnesses are at the desired places in the gates): 

- - $Z(x)f(x) - g(x)Z(\omega x) = 0$
- - $Z(\omega^0) - 1 = 0$

If there's such polynomial $T(x)$ without a remainder and the verifier is convinced, it means that all components of $P(x)$ are consistent and I really know them (I know $A(x)$, $B(x)$, $C(x)$)

#### Witness generation

$g_i = q_{L_i} * a_i + q_{R_i} * b_i + q_{M_i} * (a_ib_i) + q_{C_i} + q_{O_i} * c_i + pi_i = 0$

$\left\{ 
\begin{array}{l}
g_0 = 0 * a_0 + 0 * b_0 + 1 * (a_0b_0) + 0 + (-1) * c_0 + pi_0 = 0 & \color{Cyan} g_0 = xx\\
g_1 = 0 * a_1 + 0 * b_1 + 1 * (a_1b_1) + 0 + (-1) * c_1 + pi_1 = 0 & \color{Cyan} g_1 = xx\\
g_2 = 0 * a_2 + 0 * b_2 + 1 * (a_2b_2) + 0 + (-1) * c_2 + pi_2 = 0 & \color{Cyan} g_2 = yy\\
g_3 = 2 * a_3 + 0 * b_3 + 0 * (a_3b_3) + 0 + (-1) * c_3 + pi_3 = 0 & \color{Cyan} g_3 = 2g_0\\
g_4 = 0 * a_4 + 0 * b_4 + 1 * (a_4b_4) + 0 + (-1) * c_4 + pi_4 = 0 & \color{Cyan} g_4 = g_1g_2\\
g_5 = 1 * a_5 + (-1) * b_5 + 0 * (a_5b_5) + 0 + (-1) * c_5 + pi_5 = 0 & \color{Cyan} g_5 = g_3+g_4\\
g_6 = 1 * a_6 + 0 * b_6 + 0 * (a_6b_6) + 3 + (-1) * c_6 + pi_6 = 0 & \color{Cyan} g_6 = g_5 + 5 - 2\\
\end{array}
\right.$

$\begin{matrix}
 & \color{Cyan} a & \color{Cyan} b & \color{Cyan} c & \color{Cyan} q_{L_i} & \color{Cyan} q_{R_i} & \color{Cyan} q_{M_i} & \color{Cyan} q_{C_i} & \color{Cyan} q_{O_i} & \color{Cyan} pi_i \\
\color{Cyan} g_0 & 2 & 2 & 4 & 0 & 0 & 1 & 0 & -1 & 0 \\
\color{Cyan} g_1 & 2 & 2 & 4 & 0 & 0 & 1 & 0 & -1 & 0 \\
\color{Cyan} g_2 & 3 & 3 & 9 & 0 & 0 & 1 & 0 & -1 & 0 \\
\color{Cyan} g_3 & 4 & 0 & 8 & 2 & 0 & 0 & 0 & -1 & 0 \\
\color{Cyan} g_4 & 4 & 9 & 36 & 0 & 0 & 1 & 0 & -1 & 0 \\
\color{Cyan} g_5 & 8 & 36 & -28 & 1 & -1 & 0 & 0 & -1 & 0 \\
\color{Cyan} g_6 & -28 & 3 & -25 & 1 & 0 & 0 & 5 & -1 & -2 \\
\color{Cyan} g_7 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{matrix}$

In [ ]:
a = [2, 2, 3, 4, 4, 8, p-28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, p-28, p-25]

a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)

a = to_galois_array(a, Fp)
b = to_galois_array(b, Fp)
c = to_galois_array(c, Fp)

for i in range(n):
    assert a[i]*ql[i] + b[i]*qr[i] + a[i]*b[i]*qm[i] + c[i]*qo[i] + pi[i] + qc[i] == 0, f"Gate {i} is not satisfied"

#### Round 1

Computing wire polynomials

$A'(x): \omega^i \to a_i$

$B'(x): \omega^i \to b_i$

$C'(x): \omega^i \to c_i$

Adding blinding polynomials:

Pick random $b_0, b_1, ..., b_5$

$A(x) = A'(x) + (b_0x + b_1) \cdot Z_H(x)$

$B(x) = B'(x) + (b_2x + b_3) \cdot Z_H(x)$

$C(x) = C'(x) + (b_4x + b_5) \cdot Z_H(x)$

---

$[A(\tau)]_{G_1} = A([\tau]_{G_1})$

$[B(\tau)]_{G_1} = B([\tau]_{G_1})$

$[C(\tau)]_{G_1} = C([\tau]_{G_1})$

$\pi_1: [A(\tau)_{G_1}, B(\tau)_{G_1}, C(\tau)_{G_1}]$

In [ ]:
random_b = [Fp.Random() for i in range(0, 9)]

# blinding polynomials
bA = galois.Poly(random_b[:2], field=Fp)
bB = galois.Poly(random_b[2:4], field=Fp)
bC = galois.Poly(random_b[4:6], field=Fp)

# wire polynomials
_A = to_poly(roots, a, Fp) # degree n-1
_B = to_poly(roots, b, Fp) # degree n-1
_C = to_poly(roots, c, Fp) # degree n-1

# Wire polynomials with blinding factor
A = _A + bA*Zh # degree n+1
B = _B + bB*Zh # degree n+1
C = _C + bC*Zh # degree n+1

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + c(x)*qo(x) + qc(x) + pi(x)
G = A*QL + B*QR + A*B*QM + C*QO + QC + PI # degree 3n + 1

for i in range(0, len(roots)):
    assert G(roots[i]) == 0, f"G({roots[i]}) != 0"

assert G % Zh == 0, "G(x) % Zh(x) != 0"

# commitments [a], [b], [c]
round1 = [A(tau), B(tau), C(tau)]

#### Round 2

Computing permutation polynomial

Build polynomials $f(x)$, $g(x)$ that interpolate the following values:

$f(x): \omega^i \to ({a_i + \beta \omega^i + \gamma})({b_i + \beta k_1 \omega^i + \gamma})({c_i + \beta k_2 \omega^i + \gamma})$

$g(x): \omega^i \to ({a_i + \beta S_{\sigma 1}(\omega^i) + \gamma})({b_i + \beta S_{\sigma 2}(\omega^i) + \gamma})({c_i + \beta S_{\sigma 3}(\omega^i) + \gamma})$

Build polynomial $Z'(x)$ that interpolates the following values:

$Z'(x): \omega^i \to \left\{ 1, ..., \prod_{j=0}^{k-2}\frac{({a_j + \beta \omega^j + \gamma})({b_j + \beta k_1 \omega^j + \gamma})(c_j + \beta k_2 \omega^j + \gamma)}{({a_j + \beta S_{\sigma 1}(\omega^j) + \gamma})({b_j + \beta S_{\sigma 2}(\omega^j) + \gamma})({c_j + \beta S_{\sigma 3}(\omega^j) + \gamma})} \right\}$

Adding blinding polynomial:

Pick random $b_6, b_7, b_8$

$Z(x) = (b_6x^2 + b_7x + b_8) \cdot Z_H(x) + Z'(x)$

$Z(x)$ can be represented as a vector of coefficients:

$Z(x) = \sum_{i=0}^{k-1}z_i x^i = \left\langle z_0, z_1, ..., z_{k-1} \right\rangle$

Build polynomial $Z(\omega x)$:

$Z(\omega x) = \sum_{i=0}^{k-1} z_i\omega \cdot x^i = \left\langle z_0, \omega z_1, \omega^2 z_2, ..., \omega^{k-1}z_{k-1} \right\rangle$

---

Constraints:

$Z(\omega^0) = 1$

$Z(h)f(h) = g(h)Z(\omega h), h \in H$

*Details in plonk_components.ipynb

---

$[Z(\tau)]_{G_1} = Z([\tau]_{G_1})$

$\pi_2: [Z(\tau)]_{G_1}$

In [ ]:
def shift_poly(poly: galois.Poly, omega: galois.FieldArray):
    coeffs = poly.coefficients(order="asc")
    coeffs = [c * omega**i for i, c in enumerate(coeffs)]
    return galois.Poly(coeffs, order="asc", field=poly.field)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)

_F = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma) # degree 3(n+1)
_G = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma) # degree 3(n+1)

acc_eval = [Fp(1)]
for i in range(0, n):
    acc_eval.append(acc_eval[-1] * (_F(roots[i]) / _G(roots[i])))

assert acc_eval.pop() == Fp(1)

# differs from the original paper (shifted by x axis by -1 since all related values are)
ACC = galois.lagrange_poly(roots, Fp(acc_eval)) # degree n-1

bZ = galois.Poly(random_b[6:9], field=Fp)
Z = bZ * Zh + ACC # degree n+2

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

Z_omega = shift_poly(Z, omega=omega)

round2 = [Z(tau)]

#### Round 3

The goal is to construct the quotient polynomial confirming that:

1) All gates are correct:

$P_1(x) = A(x)Q_L(x)+B(x)Q_R(x) + A(x)B(x)Q_M(x) + C(x)Q_O(x) + Q_C(x) + PI(x)$

If $Z_H(x)$ divides $P_1(x)$ without a remainder, it means:

$g_i = a_iq_{L_i} + b_iq_{R_i} + a_ib_iq_{M_i} + c_iq_{O_i} + q_{C_i} = 0, i \in [n]$

2. All permutations are correct: 

$P_2(x) = Z(x)*f(x)-g(x)*Z(\omega x)$

If $Z_H(x)$ divides $P_2(x)$ without a remainder, it means:

$Z(x) * f(x) = g(x) * Z(\omega x)$

3. The permutation polynomial:

$L_1(x) = \left\{ 
\begin{array}{l}
1 & x = \omega^0 \\
0 & x \in {\omega^i}, i \in {1..n-1} \\
\end{array}
\right.$

$P_3(x) = (Z(x) - 1) * L_1(x)$

If $Z_H(x)$ divides $P_3(x)$ without a remainder, it means:

$Z(\omega^0) = 1$

---

$P(x) = P_0(x) + \alpha P_1(x) + \alpha^2 P_2(x)$

$P(x) = T(x)Z_H(x)$

$T(x) = \frac{P(x)}{Z_H(x)}$

Splitting $T(x)$:

$T = \sum_{i=0}^{3n+4}t_ix^i = \left\langle t_i \right\rangle_{i\in[3n+5]}$

$T_l(x) = \sum_{i=0}^{n-1}t_ix^i = \left\langle t_0, t_1, ..., t_{n-1} \right\rangle_{[n]}$

$T_m(x) = \sum_{i=0}^{n-1}t_{n + i}x^{i} = \left\langle t_{n}, t_{n+1}, ..., t_{2n - 1} \right\rangle_{[n]}$

$T_h(x) = \sum_{i=0}^{n+4}t_{2n + i}x^i = \left\langle t_{2n}, t_{2n+1}, ..., t_{3n + 4} \right\rangle_{[n+5]}$

$\color{Cyan} T_l + x^nT_m + x^{2n}T_h = \sum_{i=0}^{n-1}t_ix^i + \sum_{i=0}^{n-1}t_{n + i}x^{n + i} + \sum_{i=0}^{n+4}t_{2n + i}x^{2n + i} = T$

---

Pick random blinding scalars $b_{10}, b_{11}$

Define $T'_l(x), T'_m(x), T'_h(x)$:

$T'_l(x) = T_l(x) + b_{10}x^n$

$T'_m(x) = T_m(x) - b_{10} + b_{11}x^n$

$T'_h(x) = T_h(x) - b_{11}$

$\color{Cyan}T'_l(x) + x^nT'_m(x) + x^{2n}T'_h(x) = T_l(x) + x^nb_{10} + x^nT_m(x) - x^nb_{10} + x^{2n}b_{11} + x^{2n}T_h(x) - x^{2n}b_{11}) = T_l(x) + x^nT_m(x) + x^{2n}T_h(x) = T$

---

$[T_l(\tau)]_{G_1} = T'_l([\tau]_{G_1})$

$[T_m(\tau)]_{G_1} = T'_m([\tau]_{G_1})$

$[T_h(\tau)]_{G_1} = T'_h([\tau]_{G_1})$

$\pi_3: {[T_l(\tau)]_{G_1}, [T_h(\tau)]_{G_1}, [T_m(\tau)]_{G_1}}$


In [ ]:
alpha = numbers_to_hash(round1 + round2, Fp)

L1 = galois.lagrange_poly(roots, Fp([1] + [Fp(0)] * (n - 1))) # degree n-1
for i, r in enumerate(roots):
    # make sure L1 equal 1 at root[0] and 0 for the rest
    assert L1(r) == (Fp(1) if i == 0 else Fp(0))

P1 = G
assert P1 % Zh == 0, "P1(x) % Zh(x) != 0"

P2 = (_F * Z - _G * Z_omega)
assert P2 % Zh == 0, "P2(x) % Zh(x) != 0"

P3 = (Z - galois.Poly([1], field=Fp)) * L1
assert P3 % Zh == 0, "P3(x) % Zh(x) != 0"

P = P1 + alpha * P2 + alpha**2 * P3
assert P % Zh == 0, "P(x) % Zh(x) != 0"

for r in roots:
    assert P(r) == 0, f"T({r}) != 0"

# compute quotient polynomial
T = P // Zh # degree 3n + 5

t_coeffs = T.coefficients(order="asc")

Tl = galois.Poly(t_coeffs[:n], order="asc", field=Fp) # degree n - 1
Tm = galois.Poly(t_coeffs[n:2*(n)], order="asc", field=Fp) # degree n - 1
Th = galois.Poly(t_coeffs[2*(n):], order="asc", field=Fp) # degree n + 5

X_n = galois.Poly.Degrees([n, 0], coeffs=[1, 0], field=Fp)
X_2n = galois.Poly.Degrees([2*(n), 0], coeffs=[1, 0], field=Fp)

# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)
assert T.degree == 3 * n + 5

b10 = Fp.Random()
b11 = Fp.Random()

Tl = Tl + b10 * X_n
Tm = Tm - b10 + b11 * X_n
Th = Th - b11
assert T == (Tl + X_n * Tm + X_2n * Th)

round3 = [Tl(tau), Tm(tau), Th(tau)]

#### Round 4

$\pi_4:$

$\bar a = A(\zeta)$

$\bar b = B(\zeta)$

$\bar c = C(\zeta)$

$\bar s_{\sigma 1} = S_{\sigma 1}(\zeta)$

$\bar s_{\sigma 2} = S_{\sigma 2}(\zeta)$

$\bar z_{\omega} = Z(\omega \zeta)$

In [ ]:
zeta = numbers_to_hash(round1 + round2 + round3, Fp)

a_zeta = A(zeta)
b_zeta = B(zeta)
c_zeta = C(zeta)
s1_zeta = S1(zeta)
s2_zeta = S2(zeta)
z_omega_zeta = Z_omega(zeta)

round4 = [a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta]

#### Round 5

Compute linearisation polynomial $r(x)$:

$r(x) = \left[ \bar a \bar b \cdot q_M(x) + \bar a \cdot q_L(x) + \bar b \cdot q_R(x) + \bar c q_O(x) + PI(\zeta) + q_C(x) \right] \\+ \alpha \left[ (\bar a + \beta \zeta + \gamma)(\bar b + k_1 \beta \zeta) + \gamma)(\bar c + k_2 \beta \zeta + \gamma) \cdot Z(x) - (\bar a + \beta \bar s_{\sigma1} + \gamma)(\bar b + \beta \bar s_{\sigma2} + \gamma)(\bar c + \beta S_{\sigma3}(x) + \gamma)\bar z_\omega \right] \\+ \alpha^2\left[ (Z(x) - 1)L_1(\zeta) \right] \\- Z_H(\zeta) \cdot (t_l(x) + \zeta^n t_m(x) + \zeta^{2n}t_h(x))\\ \color{Cyan} = \left[ P'_1(x) + \alpha P'_2(x) + P'_3(x) \right] - Z_H(\zeta) \cdot T'(x)$

Compute opening proof polynomial $W_\zeta(x)$:

$h_\zeta(x) = r(x) + \\v(a(x) - \bar a) + \\v^2(b(x) - \bar b) + \\v^3(c(x) - \bar c) + \\v^4(S_{\sigma1}(x) - \bar s_{\sigma1}) + \\v^5(S_{\sigma2}(x) - \bar s_{\sigma2})$

$W\zeta(x) = \frac{h_\zeta(x)}{x-\zeta}$

Compute opening proof polynomial $W_\zeta\omega(x)$:

$h_{\zeta\omega}(x) = Z(x) - \bar z_\omega$

$W_{\zeta\omega}(x) = \frac{h_{\zeta\omega}(x)}{x - \zeta \omega}$

---

$[W_\zeta]_{G_1} = W_\zeta([\tau]_{G_1})$

$[W_\zeta\omega]_{G_1} = W_{\zeta\omega}([\tau]_{G_1})$

$\pi_5: [W_\zeta]_{G_1}, [W_\zeta\omega]_{G_1}$

In [ ]:
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)

R = (
    QM * a_zeta * b_zeta +
    QL * a_zeta +
    QR * b_zeta +
    QO * c_zeta +
    QC + 
    PI(zeta)
)

R += (
    Z * 
    (a_zeta + beta * zeta + gamma) *
    (b_zeta + beta * zeta * k1 + gamma) *
    (c_zeta + beta * zeta * k2 + gamma) * alpha
)

R -= (
    z_omega_zeta *
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) * 
    (c_zeta + beta * S3 + gamma) * alpha
)

R += (Z - Fp(1)) * L1(zeta) * alpha**2

R -= Zh(zeta) * (Tl + zeta**n * Tm + zeta**(2*n) * Th)

W_zeta = (
    R +
    (A - a_zeta) * v +
    (B - b_zeta) * v**2 +
    (C - c_zeta) * v**3 +
    (S1 - s1_zeta) * v**4 +
    (S2 - s2_zeta) * v**5
)

X_minus_zeta = galois.Poly([1, -zeta], field=Fp)
assert W_zeta % X_minus_zeta == 0, "Wzeta(x) % X - zeta != 0"

W_zeta = W_zeta // X_minus_zeta

X_minus_omega_zeta = galois.Poly([1, -(omega*zeta)], field=Fp)

W_omega_zeta = (Z - z_omega_zeta)
assert W_omega_zeta % X_minus_omega_zeta == 0, "W_omega_zeta(x) % X - ω*zeta != 0"

W_omega_zeta = W_omega_zeta // X_minus_omega_zeta

round5 = [W_zeta(tau), W_omega_zeta(tau)]

In [ ]:
proof = {
    "A": round1[0],
    "B": round1[1],
    "C": round1[2],
    "Z": round2[0],
    "Tl": round3[0],
    "Tm": round3[1],
    "Th": round3[2],
    "W_zeta": round5[0],
    "W_omega_zeta": round5[1],
    "a_zeta": round4[0],
    "b_zeta": round4[1],
    "c_zeta": round4[2],
    "s1_zeta": round4[3],
    "s2_zeta": round4[4],
    "z_omega_zeta": round4[5],
    "Fp": p,
}

circuit = {
    "QM": QM.coeffs,
    "QL": QL.coeffs,
    "QR": QR.coeffs,
    "QO": QO.coeffs,
    "QC": QC.coeffs,
    "PI": PI.coeffs,
    "Zh": Zh.coeffs,
    "L1": L1.coeffs,
    "S1": S1.coeffs,
    "S2": S2.coeffs,
    "S3": S3.coeffs,
    "k1": k1,
    "k2": k2,
    "tau": tau.tau,
    "Fp": p,
    "omega": omega,
    "n": n
}

### Verification

#### Circuit data preprocessing

$[Q_L(\tau)]_{G_1} = Q_L([\tau]_{G_1})$
$[Q_R(\tau)]_{G_1} = Q_R([\tau]_{G_1})$
$[Q_O(\tau)]_{G_1} = Q_O([\tau]_{G_1})$
$[Q_M(\tau)]_{G_1} = Q_M([\tau]_{G_1})$
$[Q_C(\tau)]_{G_1} = Q_C([\tau]_{G_1})$

$[S_{\sigma 1}(\tau)]_{G_1} = S_{\sigma 1}([\tau]_{G_1})$
$[S_{\sigma 2}(\tau)]_{G_1} = S_{\sigma 2}([\tau]_{G_1})$
$[S_{\sigma 3}(\tau)]_{G_1} = S_{\sigma 3}([\tau]_{G_1})$

#### Proof unpacking

$\pi_1: [A(\tau)]_{G_1}, [B(\tau)]_{G_1}, [C(\tau)]_{G_1}$
$\pi_2: [Z(\tau)]_{G_1}$
$\pi_3: [T_l(\tau)]_{G_1}, [T_m(\tau)], T_h(\tau)$
$\pi_4: \bar a, \bar b, \bar c, \bar s_{\sigma 1}, \bar s_{\sigma 2}, \bar z_{\omega}$
$\pi_5: [W_{\zeta}(\tau)]_{G_1}, W_{\omega \zeta}(\tau)_{G_1}$

#### Deriving random values

todo

#### Evaluating polynomials

$L_1(\zeta) = \frac{\omega (\zeta^n - 1)}{n (\zeta - \omega)}$ [Explanation](https://crypto.stackexchange.com/questions/107642/plonks-computation-of-the-first-lagrange-polynomial-at-zeta) - The original formula doesn't work here because in this implementation all evaluating points are shifted and start from $\omega^0$

$Z_H(\zeta) = \zeta^n - 1$

$PI(\zeta)$

In [ ]:
# These evaluations are calculated beforehand during the setup phase
qm_tau = QM(tau)
ql_tau = QL(tau)
qr_tau = QR(tau)
qo_tau = QO(tau)
qc_tau = QC(tau)
s1_tau = S1(tau)
s2_tau = S2(tau)
s3_tau = S3(tau)

a_tau, b_tau, c_tau = round1

z_tau, = round2

tl_tau, tm_tau, th_tau = round3

a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta = round4

w_zeta_tau, w_omega_zeta_tau = round5

validate_point(qm_tau)
validate_point(ql_tau)
validate_point(qr_tau)
validate_point(qo_tau)
validate_point(qc_tau)
validate_point(z_tau)
validate_point(s1_tau)
validate_point(s2_tau)
validate_point(s3_tau)
validate_point(tl_tau)
validate_point(tm_tau)
validate_point(th_tau)
validate_point(a_tau)
validate_point(b_tau)
validate_point(c_tau)
validate_point(w_zeta_tau)
validate_point(w_omega_zeta_tau)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)
alpha = numbers_to_hash(round1 + round2, Fp)
zeta = numbers_to_hash(round1 + round2 + round3, Fp)
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)

# evaluate vanishing and L1 polynomial ar zeta
Zh_zeta = Zh(zeta)

L1 = galois.Poly([1], field=Fp)
for i in range(1, n):
    L1 *= galois.Poly([1, -omega**i], field=Fp) // (Fp(1) - omega**i)

L1_zeta = L1(zeta)
# L1_zeta = Fp((omega * (zeta ** n - Fp(1))) // (n * (zeta - omega)))
pi_zeta = PI(zeta)

#### Idea

$r_0 = PI(\zeta) - \alpha^2 L_1(\zeta) - \alpha (\bar a + \beta \bar S_{\sigma 1} + \gamma)(\bar b + \beta \bar S_{\sigma 2} + \gamma)(\bar c + \gamma)\bar z_\omega$

$r(x) = \left[ \bar a \bar b \cdot q_M(x) + \bar a \cdot q_L(x) + \bar b \cdot q_R(x) + \bar c q_O(x) + PI(\zeta) + q_C(x) \right] + \\ \alpha \left[ (\bar a + \beta \zeta + \gamma)(\bar b + k_1 \beta \zeta) + \gamma)(\bar c + k_2 \beta \zeta + \gamma) \cdot Z(x) - (\bar a + \beta \bar s_{\sigma1} + \gamma)(\bar b + \beta \bar s_{\sigma2} + \gamma)(\bar c + \beta S_{\sigma3}(x) + \gamma)\bar z_\omega \right] + \\ \alpha^2\left[ (Z(x) - 1)L_1(\zeta) \right] - \\ Z_H(\zeta) \cdot (t_l(x) + \zeta^n t_m(x) + \zeta^{2n}t_h(x))$

$r'(x) = r(x) - r_0 =$

$\left[ \bar a \bar b \cdot q_M(x) + \bar a \cdot q_L(x) + \bar b \cdot q_R(x) + \bar c q_O(x) + q_C(x) \right] + 
\\ \alpha \left[ (\bar a + \beta \zeta + \gamma)(\bar b + k_1 \beta \zeta) + \gamma)(\bar c + k_2 \beta \zeta + \gamma) \cdot Z(x) - (\bar a + \beta \bar s_{\sigma1} + \gamma)(\bar b + \beta \bar s_{\sigma2} + \gamma)\beta S_{\sigma3}(x)\bar z_\omega \right] \\ + \alpha^2Z(x)L_1(\zeta) - \\ Z_H(\zeta) \cdot (t_l(x) + \zeta^n t_m(x) + \zeta^{2n}t_h(x))$

$D(x) = r'(x) + uZ(x) =$ 

$\left[ \bar a \bar b \cdot q_M(x) + \bar a \cdot q_L(x) + \bar b \cdot q_R(x) + \bar c q_O(x) + q_C(x) \right] \\+ Z(x) \left[ u + \alpha (\bar a + \beta \zeta + \gamma)(\bar b + k_1 \beta \zeta) + \gamma)(\bar c + k_2 \beta \zeta + \gamma) + \alpha^2Z(x)L_1(\zeta) \right] + \\ \alpha \beta S_{\sigma3}(x)\bar z_\omega \left[ \bar (a + \beta \bar s_{\sigma1} + \gamma)(\bar b + \beta \bar s_{\sigma2} + \gamma) \right] - \\ Z_H(\zeta) \cdot (t_l(x) + \zeta^n t_m(x) + \zeta^{2n}t_h(x))$

$F(x) = D(x) + va(x) + v^2b(x) + v^3c(x) + v^4s_{\sigma 1}(x) + v^5s_{\sigma 2}(x)$

$E(x) = -r_0 + v\bar a + v^2\bar b + v^3\bar c + v^4\bar s_{\sigma 1} + v^5\bar s_{\sigma 2}$

$F(x) - E(x) = \left[ r'(x) + uZ(x) + ... \right] - \left[ -r_0 + ... \right] = \left[ r(x) - r_0 + r_0 + uZ(x) + ... \right] - \left[ ... \right] = \\ \left[ r(x) + v(a(x) - \bar a) + v^2(b(x) - \bar b) + v^3(c(x) - \bar c) + v^4(s_{\sigma 1}(x) - \bar s_{sigma 1}) + v^5 (s_{\sigma 2}(x) - \bar s_{\sigma 2}) \right] + \left[ uZ(x) \right] = \color{Cyan} h_{\zeta}(x) + uh_{\zeta \omega}(x)$

---

$\tau(W_\zeta(\tau) + uW_{\zeta\omega}(\tau)) = \zeta W_\zeta(\tau) + u\zeta\omega W_{\zeta\omega}(\tau) + F(\tau) - E(\tau)$

$\tau W_\zeta(\tau) + \tau u W_{\zeta\omega}(\tau) - \zeta W_\zeta(\tau) - u\zeta\omega W_{\zeta\omega}(\tau) - F(\tau) + E(\tau) = 0$

$(\tau - \zeta)W_\zeta(\tau) + u(\tau - \zeta\omega)W_{\zeta\omega}(\tau) - F(\tau) + E(\tau) = 0$

$(\tau - \zeta)\frac{h_\zeta(\tau)}{\tau - \zeta} + u(\tau - \zeta\omega)\frac{h_{\zeta\omega}(\tau)}{\tau - \zeta\omega} - (h_\zeta(\tau) + uh_{\zeta\omega}(\tau)) = 0$

$h_\zeta(\tau) + uh_{\zeta\omega}(\tau) - h_\zeta(\tau) - uh_{\zeta\omega}(\tau) = 0 \implies 0=0$

#### Implementation

$r_0 = PI(\zeta) - \alpha^2 L_1(\zeta) - \alpha (\bar a + \beta \bar S_{\sigma 1} + \gamma)(\bar b + \beta \bar S_{\sigma 2} + \gamma)(\bar c + \gamma)\bar z_\omega$

$[D]_{G_1} = \left[ \bar a \bar b [q_M]_{G_1} + \bar a [q_L]_{G_1} + \bar b [q_R]_{G_1} + \bar c [q_O]_{G_1} + [q_C]_{G_1} \right] + \\ [Z(\tau)]_{G_1} \cdot \left[ u + \alpha(\bar a + \beta\zeta + \gamma)(\bar b + \beta k_1 \zeta + \gamma)(\bar c + \beta k_2 \zeta + \gamma) + \alpha^2L_1(\zeta) \right] - [s_{\sigma 3}]_{G_1} \cdot \left[ \alpha\beta\bar z_{\omega}(\bar a + \beta \bar s_{\sigma 1} + \gamma)(\bar b + \beta \bar s_{\sigma 2} + \gamma) \right] - \\ Z_H(\zeta) \cdot \left( [t_l]_{G_1} + \zeta^n[t_m]_{G_1} + \zeta^{2n}[t_h]_{G_1} \right)$

$[F]_{G_1} = [D]_{G_1} + u[A(\tau)]_{G_1} + u^2[B(\tau)]_{G_1} + u^3[C(\tau)]_{G_1} + u^4[S_{\sigma 1}]_{G_1} + u^5[S_{\sigma 2}]_{G_1}$

$[E]_{G_1} = (-r_0 + u\bar a + u^2\bar b + u^3\bar c + u^4\bar s_{\sigma 1} + u^5 \bar s_{\sigma 2} + u\bar z_{\omega}) \cdot [1]_{G_1}$

---

$e([W_\zeta]_{G_1} + u \cdot [W_{\zeta\omega}]_{G_1}, [\tau]_{G_2}) = e(\zeta \cdot [W_{\zeta}]_{G_1} + u\zeta\omega \cdot [W_{\zeta\omega}]_{G_1} + [F]_{G_1} - [E]_{G_1}, [1]_{G_2})$

$[\tau(W_\zeta + uW_{\zeta\omega})]_{G_T} = [\zeta \cdot W_{\zeta} + u\zeta\omega \cdot W_{\zeta\omega} + F - E]_{G_T}$

$\tau(W_\zeta + uW_{\zeta\omega}) = \zeta \cdot W_{\zeta} + u\zeta\omega \cdot W_{\zeta\omega} + F - E$

In [ ]:
# Compute constant part of r polynomials,
# r0 = PI_z - L1_z ...
r0 = (
	pi_zeta - L1_zeta * alpha**2 -
	(a_zeta + beta * s1_zeta + gamma) *
	(b_zeta + beta * s2_zeta + gamma) *
	(c_zeta + gamma) * z_omega_zeta * alpha
)

# Compute first part of batched polynomial commitment 
D_tau = (
	qm_tau * a_zeta * b_zeta +
	ql_tau * a_zeta +
	qr_tau * b_zeta +
	qo_tau * c_zeta +
	qc_tau
)

D_tau += (
    z_tau * (
		(a_zeta + beta * zeta + gamma) * 
        (b_zeta + beta * zeta * k1 + gamma) * 
        (c_zeta + beta * zeta * k2 + gamma) * alpha + 
        L1_zeta * alpha**2 + u
    )
)

D_tau -= (
	s3_tau * (a_zeta + beta * s1_zeta + gamma) * 
    (b_zeta + beta * s2_zeta + gamma) * 
    alpha * beta * z_omega_zeta
)

D_tau -= (
    (tl_tau + tm_tau * zeta**n + th_tau * zeta**(2*n)) * Zh_zeta
)

F_tau = (
    D_tau + 
	a_tau * v +
	b_tau * v**2 +
	c_tau * v**3 +
	s1_tau * v**4 +
	s2_tau * v**5
)

E_tau = (
	-r0 +
	v * a_zeta +
	v**2 * b_zeta +
	v**3 * c_zeta +
	v**4 * s1_zeta +
	v**5 * s2_zeta +
	u * z_omega_zeta
)

E_tau = G1 * E_tau

e1 = w_zeta_tau + w_omega_zeta_tau * u
e2 = w_zeta_tau * zeta + w_omega_zeta_tau * (u * zeta * omega) + F_tau + (E_tau * Fp(p-1))

assert tau.tau2.pair(e1) == G2.pair(e2), "wrong pairing"